# bugs crawler(0603_genre_ver)_d1,4,5

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import display
import pandas as pd
import datetime as dt

import os
os.chdir('/content/drive/MyDrive/21-1')

In [ ]:
## 0.start : 벅스 뮤직 PD로 접속
req = requests.get('https://music.bugs.co.kr/musicpd?wl_ref=M_left_02_04')
html = req.content
soup = BeautifulSoup(html, 'lxml') 

## 1.뮤직스타일 링크 추출 -> dict으로 저장
dict_categorytheme={}
for a_tag in soup.select('ul.listCategoryTheme')[0].select('a.hyrend'):
    dict_categorytheme[a_tag.get_text().rstrip()]=a_tag['href'] #장르 오른쪽 공백 지우기
dict_categorytheme

{'감각적인': 'https://music.bugs.co.kr/musicpd?tag_id=9895',
 '감성적인': 'https://music.bugs.co.kr/musicpd?tag_id=5432',
 '강한': 'https://music.bugs.co.kr/musicpd?tag_id=5499',
 '걸그룹': 'https://music.bugs.co.kr/musicpd?tag_id=1505',
 '공연/라이브': 'https://music.bugs.co.kr/musicpd?tag_id=5788',
 '그루브한': 'https://music.bugs.co.kr/musicpd?tag_id=5615',
 '달달한': 'https://music.bugs.co.kr/musicpd?tag_id=4184',
 '듀엣/피처링': 'https://music.bugs.co.kr/musicpd?tag_id=5707',
 '목소리/음색': 'https://music.bugs.co.kr/musicpd?tag_id=18797',
 '몽환적인': 'https://music.bugs.co.kr/musicpd?tag_id=1377',
 '발라드한': 'https://music.bugs.co.kr/musicpd?tag_id=6128',
 '섹시한': 'https://music.bugs.co.kr/musicpd?tag_id=8015',
 '소울풀한': 'https://music.bugs.co.kr/musicpd?tag_id=14275',
 '신나는': 'https://music.bugs.co.kr/musicpd?tag_id=429',
 '쓸쓸한': 'https://music.bugs.co.kr/musicpd?tag_id=4182',
 '아이돌': 'https://music.bugs.co.kr/musicpd?tag_id=2049',
 '애절한': 'https://music.bugs.co.kr/musicpd?tag_id=5561',
 '어쿠스틱한': 'https://music.bugs.co.

In [ ]:
keys1 = ('신나는', '발라드한', '그루브한', '감성적인')
d1 = {k: dict_categorytheme[k] for k in keys1}
keys2 = ('어쿠스틱한', '일렉트로닉', '달달한', '몽환적인')
d2 = {k: dict_categorytheme[k] for k in keys2}
keys3 = ('강한', '잔잔한', '올디스', '애절한')
d3 = {k: dict_categorytheme[k] for k in keys3}
keys4 = ('감각적인', '섹시한', '쓸쓸한', '소울풀한')
d4 = {k: dict_categorytheme[k] for k in keys4}
keys5 = ('청량한', '듀엣/피처링', '목소리/음색', '걸그룹', '아이돌', '공연/라이브')
d5 = {k: dict_categorytheme[k] for k in keys5}

In [ ]:
d1

{'감성적인': 'https://music.bugs.co.kr/musicpd?tag_id=5432',
 '그루브한': 'https://music.bugs.co.kr/musicpd?tag_id=5615',
 '발라드한': 'https://music.bugs.co.kr/musicpd?tag_id=6128',
 '신나는': 'https://music.bugs.co.kr/musicpd?tag_id=429'}

### d1

In [ ]:
dict_category = {'category_theme':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d1.items():
    print(name_categorytheme,url_categorytheme)
    req_category = requests.get(url_categorytheme)
    html_category = req_category.content
    soup_category = BeautifulSoup(html_category, 'lxml')

    total_num = soup_category.select_one('p.desc').get_text()
    total_num = int(re.sub(r'\D', '', total_num))
    page_num = total_num//24 + 2

    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]

    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        req_pages = requests.get(url_categorypages)
        html_pages = req_pages.content
        soup_pages = BeautifulSoup(html_pages, 'lxml')

        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']

        for url_categoryalbum in dict_categoryalbum.values():
            req_album = requests.get(url_categoryalbum)
            html_album = req_album.content
            soup_album = BeautifulSoup(html_album, 'lxml') 
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}):
                dict_category['category_theme'].append(name_categorytheme)
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)

        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(1))

신나는 https://music.bugs.co.kr/musicpd?tag_id=429
20번째 페이지 앨범480개


,category_theme,artist_disp_nm,track_title,album_style
10952,신나는,Alpaca Sports(알파카 스포츠),I'll Do Anything You Want,#해외#해외 팝#팝송#나들이/소풍#기분전환#경쾌한#비타민송#드라이브#상쾌한#청량한#...


40번째 페이지 앨범960개


,category_theme,artist_disp_nm,track_title,album_style
21850,신나는,Sigrid(시그리드),Mine Right Now (High Contrast Remix),#해외#리믹스#일렉트로닉#일렉트로닉팝#신나는#편집숍/매장#클럽/파티#루프탑#술집/펍


60번째 페이지 앨범1440개


...

,category_theme,artist_disp_nm,track_title,album_style
188787,감성적인,어쿠루브(Acourve),그날,#국내#인디#감성적인#남성보컬#카페#밤/새벽#저녁#잔잔한#출근/퇴근길#목소리/음색#...


100번째 페이지 앨범2400개


,category_theme,artist_disp_nm,track_title,album_style
197098,감성적인,Jayme Dee(제이미 디),Till I Fall Asleep,#팝송#카페#감성적인#비/흐림#스테디셀러#해외 팝


120번째 페이지 앨범2880개


,category_theme,artist_disp_nm,track_title,album_style
205974,감성적인,Kei Kobayashi(고바야시 케이/小林桂),I Fall In Love Too Easily,#잔잔한#가을#카페#감성적인#밤/새벽#가을밤


In [ ]:
pd.DataFrame.from_dict(dict_category).shape

(207997, 4)

In [ ]:
display(pd.DataFrame.from_dict(dict_category))

,category_theme,artist_disp_nm,track_title,album_style
0,신나는,Charlotte Sands,Dress,#국내외#노동요#내적댄스#신나는#트렌디한#경쾌한#기분전환#드라이브#출근/퇴근길#화창...
1,신나는,OPO,See You Through,#국내외#노동요#내적댄스#신나는#트렌디한#경쾌한#기분전환#드라이브#출근/퇴근길#화창...
2,신나는,Maori,Did It for You,#국내외#노동요#내적댄스#신나는#트렌디한#경쾌한#기분전환#드라이브#출근/퇴근길#화창...
3,신나는,Foley,Rendezvous,#국내외#노동요#내적댄스#신나는#트렌디한#경쾌한#기분전환#드라이브#출근/퇴근길#화창...
4,신나는,Disco Despair(디스코 디스페어),Keep Hiding (Feat. Sheare),#국내외#노동요#내적댄스#신나는#트렌디한#경쾌한#기분전환#드라이브#출근/퇴근길#화창...
...,...,...,...,...
207992,감성적인,못(Mot),서울은 흐림 (feat. 한희정),#오후#감성적인
207993,감성적인,소규모 아카시아 밴드,슬픈 사랑 노래,#오후#감성적인
207994,감성적인,어른아이(Adultchild),민들레 (Dandelion),#오후#감성적인
207995,감성적인,이다오,등대지기,#오후#감성적인


In [ ]:
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_total_genre_d1.csv', index = False, encoding = 'UTF-8')

###d4

In [ ]:
## 2. 뮤직스타일 링크로 삽입 
dict_category = {'category_theme':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d4.items():
    #print(name_categorytheme,url_categorytheme) # : 22개 장르 모두 나오는 것 확인
    ### 2-1. 뮤직스타일 링크에서 각 페이지 불러오기 : name_categorytheme 장르에서만!!
    req_category = requests.get(url_categorytheme)
    html_category = req_category.content
    soup_category = BeautifulSoup(html_category, 'lxml')
    #print(soup_category)

    ### 2-2. page iteration
    total_num = soup_category.select_one('p.desc').get_text()
    total_num = int(re.sub(r'\D', '', total_num))
    page_num = total_num//24 + 2

    ### 몇 페이지인지
    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]
    #print('으악',base_url)
    album_count=0 #앨범갯수 맞추기용 상수

    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        req_pages = requests.get(url_categorypages)
        html_pages = req_pages.content
        soup_pages = BeautifulSoup(html_pages, 'lxml')

        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        #print('dict_categoryalbum',dict_categoryalbum) # 순서대로는 아닌데 쨋든 album dict 완료 
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1 #앨범갯수 맞추기용 상수
            req_album = requests.get(url_categoryalbum)
            html_album = req_album.content
            soup_album = BeautifulSoup(html_album, 'lxml') # pip install lxml
            ## 앨범 특징 : 팀원 피드백) 곡 제목-스타일-음악에 대표되는 감성? 을 위주로 중 음악에 대표되는 감성을 앨범 #로 태그해놔서 가져와봤어요
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_theme'].append(name_categorytheme)
            # 수작업으로 이렇게 했는데 추후에 dict에서 key 빼서 할 수도 있을 것같아요. 근데 22개 장르라 그냥 수작업으로 해도 괜찮을 것같고 그렇습니다아 category
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)

        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(2))

20번째 페이지 앨범480개


,category_theme,artist_disp_nm,track_title,album_style
10608,감각적인,KWAYE,Cool Kids,#해외#알앤비#그루브한#감각적인#저녁#밤/새벽#트렌디한#힙스터#밤샘작업#작업실
10609,감각적인,Harrison Brome,9 To 5,#해외#알앤비#그루브한#감각적인#저녁#밤/새벽#트렌디한#힙스터#밤샘작업#작업실


40번째 페이지 앨범960개


,category_theme,artist_disp_nm,track_title,album_style
20321,감각적인,Labrinth(라브린느),Jealous (Bakermat Remix),#해외#출근/퇴근길#감각적인#등교/출근길#연휴병#월요병
20322,감각적인,Kwabs,Fight For Love,#해외#출근/퇴근길#감각적인#등교/출근길#연휴병#월요병


In [ ]:
pd.DataFrame.from_dict(dict_category).shape

(35204, 4)

In [ ]:
display(pd.DataFrame.from_dict(dict_category))

,category_theme,artist_disp_nm,track_title,album_style
0,감각적인,이더(E the),City (Feat. Sllo),#국내#인디#국내 인디#밤/새벽#감각적인#알앤비
1,감각적인,이고도,Mouse,#국내#인디#국내 인디#밤/새벽#감각적인#알앤비
2,감각적인,버디(BUDY),Fly High,#국내#인디#국내 인디#밤/새벽#감각적인#알앤비
3,감각적인,태윤(TAEYOON),You,#국내#인디#국내 인디#밤/새벽#감각적인#알앤비
4,감각적인,Sofa(소파),Dreams,#국내#인디#국내 인디#밤/새벽#감각적인#알앤비
...,...,...,...,...
35199,소울풀한,The Jacksons(잭슨스),Show You the Way to Go,#올디스#해외#소울풀한
35200,소울풀한,Manhattans(맨하탄스),There's No Me Without You,#올디스#해외#소울풀한
35201,소울풀한,The Hues Corporation,Rock the Boat,#올디스#해외#소울풀한
35202,소울풀한,Lou Rawls(루 롤스),You'll Never Find Another Love Like Mine,#올디스#해외#소울풀한


In [ ]:
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_total_genre_d4.csv', index = False, encoding = 'UTF-8')

### d5

In [ ]:
dict_category = {'category_theme':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d5.items():
    soup_category = BeautifulSoup(requests.get(url_categorytheme).content, 'lxml')
    page_num = int(re.sub(r'\D', '', soup_category.select_one('p.desc').get_text()))//24 + 2
    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]
    album_count=0
    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        soup_pages = BeautifulSoup(requests.get(url_categorypages).content, 'lxml')
        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1
            soup_album = BeautifulSoup(requests.get(url_categoryalbum).content, 'lxml')
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_theme'].append(name_categorytheme)
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)
        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(2))
pd.DataFrame.from_dict(dict_category).shape
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('/content/drive/MyDrive/21-1/data_total_genre_d5.csv', index = False, encoding = 'UTF-8')

20번째 페이지 앨범480개


,category_theme,artist_disp_nm,track_title,album_style
26825,목소리/음색,휘성(Realslow),With Me,#국내#가요#발라드한#발라드#남성보컬#감미로운#목소리/음색#국내 발라드
26826,목소리/음색,케이윌,그립고 그립고 그립다,#국내#가요#발라드한#발라드#남성보컬#감미로운#목소리/음색#국내 발라드


20번째 페이지 앨범480개


,category_theme,artist_disp_nm,track_title,album_style
43768,아이돌,시우민(XIUMIN),Young & Free,#가요#국내#아이돌#유닛
43769,아이돌,펜타곤,"You Are (진호, 후이, 홍석, 신원, 여원, 옌안, 키노)",#가요#국내#아이돌#유닛


In [ ]:
display(pd.DataFrame.from_dict(dict_category))

,category_theme,artist_disp_nm,track_title,album_style
0,청량한,NCT DREAM,Chewing Gum,#국내#가요#아이돌#신나는#청량한#여름#화창한 날#드라이브#산책/여행
1,청량한,우진영,설레고 난리 (Falling in love),#국내#가요#아이돌#신나는#청량한#여름#화창한 날#드라이브#산책/여행
2,청량한,이달의 소녀,Hi High,#국내#가요#아이돌#신나는#청량한#여름#화창한 날#드라이브#산책/여행
3,청량한,신현희와김루트,PARADISE,#국내#가요#아이돌#신나는#청량한#여름#화창한 날#드라이브#산책/여행
4,청량한,Apink (에이핑크),A L R I G H T,#국내#가요#아이돌#신나는#청량한#여름#화창한 날#드라이브#산책/여행
...,...,...,...,...
54763,공연/라이브,김동률,기억의 습작,#공연/라이브#공연/페스티벌
54764,공연/라이브,Avril Lavigne(에이브릴 라빈),"Girlfriend (Live at The Roxy Theatre, Los Ange...",#공연/라이브#공연/페스티벌
54765,공연/라이브,Muse(뮤즈),Starlight,#공연/라이브#공연/페스티벌
54766,공연/라이브,싸이 (PSY),아버지,#공연/라이브#공연/페스티벌


# bugs crawler(0603_genre_ver)_d2,3

In [24]:
from google.colab import drive 
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/21-1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import display
import pandas as pd
import datetime as dt

In [26]:
## 0.start : 벅스 뮤직 PD로 접속
req = requests.get('https://music.bugs.co.kr/musicpd?wl_ref=M_left_02_04')
html = req.content
soup = BeautifulSoup(html, 'lxml') # pip install lxml

## 1.뮤직스타일 링크 추출 -> dict으로 저장
dict_categorytheme={}
for a_tag in soup.select('ul.listCategoryTheme')[0].select('a.hyrend'):
    dict_categorytheme[a_tag.get_text().rstrip()]=a_tag['href'] #장르 오른쪽 공백 지우기
dict_categorytheme

{'감각적인': 'https://music.bugs.co.kr/musicpd?tag_id=9895',
 '감성적인': 'https://music.bugs.co.kr/musicpd?tag_id=5432',
 '강한': 'https://music.bugs.co.kr/musicpd?tag_id=5499',
 '걸그룹': 'https://music.bugs.co.kr/musicpd?tag_id=1505',
 '공연/라이브': 'https://music.bugs.co.kr/musicpd?tag_id=5788',
 '그루브한': 'https://music.bugs.co.kr/musicpd?tag_id=5615',
 '달달한': 'https://music.bugs.co.kr/musicpd?tag_id=4184',
 '듀엣/피처링': 'https://music.bugs.co.kr/musicpd?tag_id=5707',
 '목소리/음색': 'https://music.bugs.co.kr/musicpd?tag_id=18797',
 '몽환적인': 'https://music.bugs.co.kr/musicpd?tag_id=1377',
 '발라드한': 'https://music.bugs.co.kr/musicpd?tag_id=6128',
 '섹시한': 'https://music.bugs.co.kr/musicpd?tag_id=8015',
 '소울풀한': 'https://music.bugs.co.kr/musicpd?tag_id=14275',
 '신나는': 'https://music.bugs.co.kr/musicpd?tag_id=429',
 '쓸쓸한': 'https://music.bugs.co.kr/musicpd?tag_id=4182',
 '아이돌': 'https://music.bugs.co.kr/musicpd?tag_id=2049',
 '애절한': 'https://music.bugs.co.kr/musicpd?tag_id=5561',
 '어쿠스틱한': 'https://music.bugs.co.

In [27]:
# slicing
keys1 = ('신나는', '발라드한', '그루브한', '감성적인')
d1 = {k: dict_categorytheme[k] for k in keys1}
keys2 = ('어쿠스틱한', '일렉트로닉', '달달한', '몽환적인')
d2 = {k: dict_categorytheme[k] for k in keys2}
keys3 = ('강한', '잔잔한', '올디스', '애절한')
d3 = {k: dict_categorytheme[k] for k in keys3}
keys4 = ('감각적인', '섹시한', '쓸쓸한', '소울풀한')
d4 = {k: dict_categorytheme[k] for k in keys4}
keys5 = ('청량한', '듀엣/피처링', '목소리/음색', '걸그룹', '아이돌', '공연/라이브')
d5 = {k: dict_categorytheme[k] for k in keys5}

### d2

In [ ]:
## 2. 뮤직스타일 링크로 삽입 
dict_category = {'category_theme':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d2.items():
    #print(name_categorytheme,url_categorytheme) # : 22개 장르 모두 나오는 것 확인
    ### 2-1. 뮤직스타일 링크에서 각 페이지 불러오기 : name_categorytheme 장르에서만!!
    req_category = requests.get(url_categorytheme)
    html_category = req_category.content
    soup_category = BeautifulSoup(html_category, 'lxml')
    #print(soup_category)

    ### 2-2. page iteration
    total_num = soup_category.select_one('p.desc').get_text()
    total_num = int(re.sub(r'\D', '', total_num))
    page_num = total_num//24 + 2

    ### 몇 페이지인지
    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]
    #print('으악',base_url)
    album_count=0 #앨범갯수 맞추기용 상수

    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        req_pages = requests.get(url_categorypages)
        html_pages = req_pages.content
        soup_pages = BeautifulSoup(html_pages, 'lxml')

        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        #print('dict_categoryalbum',dict_categoryalbum) # 순서대로는 아닌데 쨋든 album dict 완료 
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1 #앨범갯수 맞추기용 상수
            req_album = requests.get(url_categoryalbum)
            html_album = req_album.content
            soup_album = BeautifulSoup(html_album, 'lxml') # pip install lxml
            ## 앨범 특징 : 팀원 피드백) 곡 제목-스타일-음악에 대표되는 감성? 을 위주로 중 음악에 대표되는 감성을 앨범 #로 태그해놔서 가져와봤어요
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_theme'].append(name_categorytheme)
            # 수작업으로 이렇게 했는데 추후에 dict에서 key 빼서 할 수도 있을 것같아요. 근데 22개 장르라 그냥 수작업으로 해도 괜찮을 것같고 그렇습니다아 category
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)

        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(2))

20번째 페이지 앨범480개


,category_theme,artist_disp_nm,track_title,album_style
9449,어쿠스틱한,Corinne Bailey Rae(코린 베일리 래),Like A Star,#어쿠스틱한#팝송#비/흐림#해외 팝#포크/어쿠스틱
9450,어쿠스틱한,Babyface(베이비페이스),"Drama, Love & 'Lationships",#어쿠스틱한#팝송#비/흐림#해외 팝#포크/어쿠스틱


20번째 페이지 앨범480개


,category_theme,artist_disp_nm,track_title,album_style
26195,일렉트로닉,Seinabo Sey(세이나보 세이),Breathe (Lenno Remix),#해외#일렉트로닉#리믹스#기간별#트렌디한#Monthly Remix#NewSongs
26196,일렉트로닉,Shakka(샤카),Man Down (feat. AlunaGeorge) (99 Souls Remix) ...,#해외#일렉트로닉#리믹스#기간별#트렌디한#Monthly Remix#NewSongs


40번째 페이지 앨범960개


...

,category_theme,artist_disp_nm,track_title,album_style
54244,달달한,유승우,선 (feat. 우효) (45.7cm),#국내#달달한#한파/추위#겨울#포근한
54245,달달한,nokdu,Say My Name,#국내#달달한#한파/추위#겨울#포근한


40번째 페이지 앨범960개


,category_theme,artist_disp_nm,track_title,album_style
63165,달달한,헤일로(HALO),I Wanna Hold Your Hand,#가요#사랑/기쁨#달달한#화이트데이
63166,달달한,살두와 마흥이,왜 몰라 (Piano 파랑망또),#가요#사랑/기쁨#달달한#화이트데이


20번째 페이지 앨범480개


,category_theme,artist_disp_nm,track_title,album_style
79136,몽환적인,나윤선,Dancing With You,#밤/새벽#저녁#몽환적인
79137,몽환적인,Elvis Costello(엘비스 코스텔로),Still (feat. Brodsky Quartet),#밤/새벽#저녁#몽환적인


In [ ]:
pd.DataFrame.from_dict(dict_category).shape

(83098, 4)

In [ ]:
pd.set_option('display.max_columns', 10)
display(pd.DataFrame.from_dict(dict_category))

,category_theme,artist_disp_nm,track_title,album_style
0,어쿠스틱한,Helen,Hit And Run (Acoustic Version),#해외#팝송#해외 팝#어쿠스틱한#카페#감성적인
1,어쿠스틱한,Sam Ock(샘 옥),Little Light (feat. Clara C),#해외#팝송#해외 팝#어쿠스틱한#카페#감성적인
2,어쿠스틱한,Gabe Bondoc,Stronger Than,#해외#팝송#해외 팝#어쿠스틱한#카페#감성적인
3,어쿠스틱한,Charlie Houston,Calls,#해외#팝송#해외 팝#어쿠스틱한#카페#감성적인
4,어쿠스틱한,Verzache,Messed Up,#해외#팝송#해외 팝#어쿠스틱한#카페#감성적인
...,...,...,...,...
83093,몽환적인,Enya(엔야),Storms In Africa (Remastered 2009),#휴식/명상#밤/새벽#몽환적인
83094,몽환적인,Libera(리베라),Going Home (Radio Edit),#휴식/명상#밤/새벽#몽환적인
83095,몽환적인,Sheila Ryan(쉘라 라이언),Danny Boy (대니 보이),#휴식/명상#밤/새벽#몽환적인
83096,몽환적인,Bill Douglas(빌 더글라스),The Hills of Glencar,#휴식/명상#밤/새벽#몽환적인


In [ ]:
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_total_genre_d2.csv', index = False, encoding = 'UTF-8')

### d3

In [30]:
dict_category = {'category_theme':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d3.items():
    #print(name_categorytheme,url_categorytheme) # : 22개 장르 모두 나오는 것 확인
    ### 2-1. 뮤직스타일 링크에서 각 페이지 불러오기 : name_categorytheme 장르에서만!!
    req_category = requests.get(url_categorytheme)
    html_category = req_category.content
    soup_category = BeautifulSoup(html_category, 'lxml')
    #print(soup_category)

    ### 2-2. page iteration
    total_num = int(re.sub(r'\D', '', soup_category.select_one('p.desc').get_text()))
    page_num = total_num//24 + 2

    ### 몇 페이지인지
    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]
    album_count=0 #앨범갯수 맞추기용 상수

    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        req_pages = requests.get(url_categorypages)
        html_pages = req_pages.content
        soup_pages = BeautifulSoup(html_pages, 'lxml')

        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        #print('dict_categoryalbum',dict_categoryalbum) # 순서대로는 아닌데 쨋든 album dict 완료 
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1 #앨범갯수 맞추기용 상수
            req_album = requests.get(url_categoryalbum)
            html_album = req_album.content
            soup_album = BeautifulSoup(html_album, 'lxml') # pip install lxml
            ## 앨범 특징 : 팀원 피드백) 곡 제목-스타일-음악에 대표되는 감성? 을 위주로 중 음악에 대표되는 감성을 앨범 #로 태그해놔서 가져와봤어요
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_theme'].append(name_categorytheme)
            # 수작업으로 이렇게 했는데 추후에 dict에서 key 빼서 할 수도 있을 것같아요. 근데 22개 장르라 그냥 수작업으로 해도 괜찮을 것같고 그렇습니다아 category
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)

        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(2))

20번째 페이지 앨범480개


,category_theme,artist_disp_nm,track_title,album_style
13940,잔잔한,어반자카파,6월 14일,#국내외#연주곡#잔잔한#가을#독서#힐링#휴식/명상#편안한#북카페#에스테틱#가을밤
13941,잔잔한,Yuhki Kuramoto(유키 구라모토/倉本裕基),Lake Louise (Piano & Violin),#국내외#연주곡#잔잔한#가을#독서#힐링#휴식/명상#편안한#북카페#에스테틱#가을밤


40번째 페이지 앨범960개


,category_theme,artist_disp_nm,track_title,album_style
23500,잔잔한,Crush,2411,#국내#인디#감성적인#비/흐림#밤/새벽#카페#잔잔한#나 혼자#방안에서
23501,잔잔한,김새한길,늘,#국내#인디#감성적인#비/흐림#밤/새벽#카페#잔잔한#나 혼자#방안에서


60번째 페이지 앨범1435개


,category_theme,artist_disp_nm,track_title,album_style
31857,잔잔한,Arco(아르코),Lullaby,#잔잔한#발라드#감성적인#발라드한#잠들기 전
31858,잔잔한,Maximilian Hecker(막시밀리안 헤커),Kate Moss,#잔잔한#발라드#감성적인#발라드한#잠들기 전


20번째 페이지 앨범480개


,category_theme,artist_disp_nm,track_title,album_style
44630,올디스,The Ventures(벤처스),Pipeline,#여름#드라이브#올디스#세부장르#서프뮤직#서핑#로큰롤라이프
44631,올디스,Turtles(터틀즈),Surfer Dan,#여름#드라이브#올디스#세부장르#서프뮤직#서핑#로큰롤라이프


In [31]:
pd.DataFrame.from_dict(dict_category).shape
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_total_genre_d3.csv', index = False, encoding = 'UTF-8')